In [1]:
import sys
sys.path.insert(0,'../')
from file_tools import *
from request_tools import *
from parse_tools import *
import time
from tqdm import tqdm



SRC_DIR = './data-parsed'
TGT_DIR = './data-computed'


In [6]:
def __compute_days_between_rows__(df,window=2):
    df = pd.to_datetime(df.copy())
    return df.diff(window-1).dt.days # -1 because diff() computes the difference between the current and the previous row

def __compute_cumulative_avg_stats__(df):
    return df.expanding().mean()

def __compute_expanding_std_stats__(df):
    return df.expanding().std()

def __compute_rolling_avg_stats__(df, window=5):
    return df.rolling(window).mean()

def __compute_rolling_std_stats__(df, window=5):
    return df.rolling(window).std()

def __compute_pts_total__(df):
    return df.sum(axis=1)

def __compute_pts_spread__(df):
    return df.apply(lambda x: x.iloc[0] - x.iloc[1],axis=1)

In [22]:
SRC_TGL_CSV_LIST = get_all_files_recursive(f'{SRC_DIR}/teams', file_type=('tgl_basic.csv','tgl_advanced.csv')) # get only regular season gamelog files
SRC_TGL_CSV_LIST = [x for x in SRC_TGL_CSV_LIST if 'BOS/2023' in x] # remove playoff files
SRC_TGL_CSV = SRC_TGL_CSV_LIST[0]
# TQDM_SRC_TGL_CSV_LIST = tqdm(SRC_TGL_CSV_LIST,ncols=150,dynamic_ncols=False)
# for SRC_TGL_CSV in TQDM_SRC_TGL_CSV_LIST:
#     TQDM_SRC_TGL_CSV_LIST.set_description(SRC_TGL_CSV)       
#     # Load the gamelog csv
TGL_DF = pd.read_csv(SRC_TGL_CSV,header=[0,1])
TGL_DF = TGL_DF.drop(
    columns=[
        ('Unnamed: 0_level_0','Unnamed: 0_level_1')
    ]).rename(
    columns={
        'Boxscores_html_id':'Boxscores_id',
        'Tm_html_id':'Team_id',
        'Opp_html_id':'Opp_id',
    },level=1
)
# Compute the expanding and rolling stats
TGL_MDATE_DF = TGL_DF[('Match','Date')]
TGL_DAYS_BTW_2GM = __compute_days_between_rows__(TGL_MDATE_DF,window=2).rename(('Match','Days_Btw_2GM'))
TGL_DAYS_BTW_3GM = __compute_days_between_rows__(TGL_MDATE_DF,window=3).rename(('Match','Days_Btw_3GM'))
TGL_DAYS_BTW_4GM = __compute_days_between_rows__(TGL_MDATE_DF,window=4).rename(('Match','Days_Btw_4GM'))
TGL_DAYS_BTW_GM_DF = pd.concat([TGL_DAYS_BTW_2GM,TGL_DAYS_BTW_3GM,TGL_DAYS_BTW_4GM],axis=1) 
# Compute the total points, spread points
TGL_RESULT_DF = TGL_DF['Result'].copy()
TGL_PTS_TOTAL = __compute_pts_total__(TGL_RESULT_DF[['Tm','Opp']]).rename('Pts_Total')
TGL_PTS_SPREAD = __compute_pts_spread__(TGL_RESULT_DF[['Tm','Opp']]).rename('Pts_Spread')
TGL_RESULT_DF = pd.concat([TGL_RESULT_DF,TGL_PTS_TOTAL,TGL_PTS_SPREAD],axis=1)
# Compute the expanding and rolling stats
TGL_STATS_DF = TGL_DF.drop('Match',level=0,axis=1)
TGL_STATS_DF

Result           Advanced                                           ...   
      W/L   Tm  Opp     ORtg   DRtg   Pace    FTr   3PAr    TS%  TRB%  ...   
0       1  126  117    128.3  119.2   98.2  0.341  0.427  0.668  53.7  ...  \
1       1  111  104    120.8  113.2   91.9  0.190  0.418  0.648  54.4  ...   
2       1  126  120    128.4  122.3   98.1  0.322  0.540  0.634  43.5  ...   
3       0  102  120    104.2  122.6   97.8  0.120  0.480  0.484  42.9  ...   
4       0  123  132    115.4  123.9   96.5  0.390  0.402  0.640  44.7  ...   
..    ...  ...  ...      ...    ...    ...    ...    ...    ...   ...  ...   
77      1  122  114    117.2  109.5  104.1  0.229  0.531  0.577  44.7  ...   
78      0  101  103    114.7  117.0   88.0  0.176  0.495  0.515  53.8  ...   
79      1   97   93    101.2   97.0   95.9  0.202  0.427  0.500  49.0  ...   
80      1  121  102    128.6  108.4   94.1  0.259  0.543  0.670  60.2  ...   
81      1  120  114    124.8  118.6   96.1  0.157  0.607  0.631  48.5  ...   

              Offensive Four Factors                      
   STL%  BLK%                   eFG%  TOV%  ORB% FT/FGA   
0   8.1   6.5                  0.634   9.6  18.2  0.268  \
1   4.4  16.7                  0.608  18.2  32.4  0.190   
2   5.1   9.8                  0.592   7.5  16.3  0.264   
3   5.1  10.6                  0.460   8.7  23.3  0.100   
4   8.4   3.8                  0.591  16.5  13.2  0.317   
..  ...   ...                    ...   ...   ...    ...   
77  7.7   6.0                  0.526   9.4  20.4  0.219   
78  4.5   8.2                  0.500   7.5  29.5  0.110   
79  5.2  12.3                  0.489  13.4  22.9  0.112   
80  4.3  11.3                  0.648  17.4  36.1  0.198   
81  5.2   5.2                  0.612  13.6  26.1  0.124   

   Defensive Four Factors                     
                     eFG%  TOV%  DRB% FT/FGA  
0                   0.581  13.2  88.2  0.300  
1                   0.500   8.4  73.8  0.130  
2                   0.553   7.9  71.4  0.158  
3                   0.540  11.5  68.9  0.299  
4                   0.609  12.8  76.3  0.299  
..                    ...   ...   ...    ...  
77                  0.526  15.0  71.1  0.135  
78                  0.571  10.5  83.3  0.179  
79                  0.444  10.1  74.0  0.144  
80                  0.478   6.5  76.9  0.140  
81                  0.510   7.9  68.6  0.155  

[82 rows x 21 columns]

In [23]:
TGL_STATS_AW_DF = TGL_DF[TGL_DF[('Match','H/A')] == 0].drop('Match',level=0,axis=1)
TGL_STATS_HM_DF = TGL_DF[TGL_DF[('Match','H/A')] == 1].drop('Match',level=0,axis=1)
__compute_rolling_avg_stats__(TGL_STATS_HM_DF)


Result               Advanced                                           
      W/L     Tm    Opp     ORtg    DRtg    Pace     FTr    3PAr     TS%   
0     NaN    NaN    NaN      NaN     NaN     NaN     NaN     NaN     NaN  \
4     NaN    NaN    NaN      NaN     NaN     NaN     NaN     NaN     NaN   
5     NaN    NaN    NaN      NaN     NaN     NaN     NaN     NaN     NaN   
7     NaN    NaN    NaN      NaN     NaN     NaN     NaN     NaN     NaN   
10    0.8  122.4  114.8   121.56  113.66   98.78  0.3386  0.4640  0.6436   
11    0.8  123.4  113.8   124.08  113.92   97.74  0.3148  0.4608  0.6426   
13    1.0  124.0  111.8   123.44  110.86  100.90  0.2972  0.4574  0.6304   
17    1.0  126.6  115.4   125.60  114.14  101.32  0.2966  0.4194  0.6368   
18    1.0  126.4  112.4   126.32  112.04  100.54  0.2676  0.4340  0.6426   
19    1.0  126.8  114.2   128.62  115.64   99.20  0.2848  0.4144  0.6532   
20    1.0  128.6  112.8   129.08  113.02  100.14  0.2554  0.4440  0.6664   
21    1.0  130.2  112.6   134.22  116.20   97.12  0.2504  0.4754  0.6946   
22    0.8  128.4  114.2   129.54  114.90   97.66  0.2648  0.4978  0.6774   
29    0.6  125.8  116.8   126.90  117.44   97.70  0.2808  0.5182  0.6590   
30    0.4  118.2  111.6   117.20  110.28   99.18  0.2630  0.5388  0.6104   
31    0.2  112.6  114.0   110.82  112.04   99.78  0.2976  0.5082  0.5646   
32    0.2  110.0  111.6   107.48  108.98  100.30  0.2942  0.4918  0.5292   
33    0.4  114.6  111.2   114.20  110.80  100.34  0.2886  0.4802  0.5572   
34    0.6  118.0  108.2   117.96  108.12  100.04  0.2582  0.4936  0.5716   
35    0.8  122.8  111.2   123.16  111.50   99.70  0.2594  0.4738  0.5932   
40    1.0  121.8  107.6   123.90  109.54   98.16  0.2580  0.4868  0.6044   
41    1.0  122.6  108.6   125.32  111.10   97.68  0.2610  0.4916  0.6184   
45    1.0  119.0  108.6   118.68  108.12   98.44  0.2578  0.4758  0.5734   
49    0.8  117.2  112.2   115.92  110.78   97.32  0.2532  0.4448  0.5550   
50    0.8  119.0  114.4   115.52  110.86   97.18  0.2976  0.4542  0.5634   
51    0.8  125.4  113.8   122.00  109.94   97.44  0.2712  0.4774  0.5964   
52    0.6  119.2  112.2   115.60  108.26   97.48  0.2988  0.4820  0.5798   
54    0.6  116.2  108.4   118.70  110.28   94.20  0.2696  0.4982  0.6118   
55    0.8  118.2  107.6   120.30  109.20   96.44  0.2618  0.5298  0.6372   
56    0.8  117.0  105.2   122.30  109.82   95.82  0.2280  0.5506  0.6332   
58    0.8  114.6  107.8   118.86  111.82   96.46  0.2280  0.5310  0.6208   
62    1.0  119.2  109.2   124.46  114.10   95.78  0.2172  0.5116  0.6444   
63    0.8  119.0  112.4   120.94  114.14   98.58  0.2384  0.4884  0.6258   
64    0.6  119.4  115.4   119.16  114.80   96.64  0.2460  0.4730  0.6018   
66    0.6  118.6  112.2   116.78  110.16   97.90  0.2324  0.4684  0.6010   
73    0.6  117.2  109.4   113.80  106.06   99.20  0.2514  0.4610  0.5876   
74    0.6  121.2  105.4   114.36   99.32  101.92  0.2258  0.4758  0.5946   
77    0.8  124.6  105.2   117.22   98.68  102.34  0.2240  0.5130  0.5968   
79    1.0  118.2   97.6   115.02   95.30  102.48  0.2186  0.4902  0.5892   
80    1.0  119.4   99.4   117.88   98.50  101.18  0.2250  0.4874  0.6044   
81    1.0  119.4  103.2   119.78  103.96   99.60  0.2156  0.5184  0.6122   

           ...              Offensive Four Factors                         
     TRB%  ...  STL%   BLK%                   eFG%   TOV%   ORB%  FT/FGA   
0     NaN  ...   NaN    NaN                    NaN    NaN    NaN     NaN  \
4     NaN  ...   NaN    NaN                    NaN    NaN    NaN     NaN   
5     NaN  ...   NaN    NaN                    NaN    NaN    NaN     NaN   
7     NaN  ...   NaN    NaN                    NaN    NaN    NaN     NaN   
10  49.00  ...  6.10   7.54                 0.5996  12.30  15.68  0.2788   
11  49.22  ...  5.12   7.82                 0.6016  11.70  17.44  0.2586   
13  49.76  ...  6.12   9.12                 0.5896  10.68  18.72  0.2452   
17  50.58  ...  6.10   9.16                 0.5992

In [40]:
def __compute_h2h_avg_stats__(TGL_DF):
    TGL_H2H_CUMU_AVG_STATS_DF_LIST = []
    TGL_H2H_CUMU_STD_STATS_DF_LIST = []
    for opp_id in TGL_DF[('Match','Opp_id')].unique():
        QUERY_DF_I = TGL_DF[TGL_DF[('Match','Opp_id')] == opp_id].drop('Match',level=0,axis=1)
        TGL_H2H_CUMU_AVG_STATS_DF_LIST.append(__compute_cumulative_avg_stats__(QUERY_DF_I))
        TGL_H2H_CUMU_STD_STATS_DF_LIST.append(__compute_expanding_std_stats__(QUERY_DF_I))
    TGL_H2H_CUMU_AVG_STATS_DF = pd.concat(TGL_H2H_CUMU_AVG_STATS_DF_LIST,axis=0).sort_index()
    TGL_H2H_CUMU_STD_STATS_DF = pd.concat(TGL_H2H_CUMU_STD_STATS_DF_LIST,axis=0).sort_index()
    return TGL_H2H_CUMU_AVG_STATS_DF, TGL_H2H_CUMU_STD_STATS_DF

TGL_STATS_H2H_CUMU_AVG_DF, TGL_STATS_H2H_CUMU_STD_DF = __compute_h2h_avg_stats__(TGL_DF)
TGL_STATS_H2H_CUMU_AVG_DF


Result                            Advanced                          
      W/L          Tm         Opp        ORtg        DRtg       Pace   
0    1.00  126.000000  117.000000  128.300000  119.200000  98.200000  \
1    1.00  111.000000  104.000000  120.800000  113.200000  91.900000   
2    1.00  126.000000  120.000000  128.400000  122.300000  98.100000   
3    0.00  102.000000  120.000000  104.200000  122.600000  97.800000   
4    0.00  123.000000  132.000000  115.400000  123.900000  96.500000   
..    ...         ...         ...         ...         ...        ...   
77   0.50  119.500000  116.000000  120.700000  117.400000  99.150000   
78   0.75  110.750000  106.500000  120.350000  115.850000  91.875000   
79   1.00  106.333333  102.333333  110.966667  106.800000  95.800000   
80   1.00  110.000000  102.250000  115.375000  107.200000  95.375000   
81   1.00  126.666667  113.333333  127.666667  114.266667  99.166667   

                                       ...                        
         FTr    3PAr       TS%   TRB%  ...      STL%       BLK%   
0   0.341000  0.4270  0.668000  53.70  ...  8.100000   6.500000  \
1   0.190000  0.4180  0.648000  54.40  ...  4.400000  16.700000   
2   0.322000  0.5400  0.634000  43.50  ...  5.100000   9.800000   
3   0.120000  0.4800  0.484000  42.90  ...  5.100000  10.600000   
4   0.390000  0.4020  0.640000  44.70  ...  8.400000   3.800000   
..       ...     ...       ...    ...  ...       ...        ...   
77  0.205500  0.5555  0.596500  43.20  ...  4.900000   4.800000   
78  0.197250  0.4505  0.611250  53.25  ...  5.325000   8.025000   
79  0.203667  0.4130  0.554333  50.20  ...  6.966667   9.833333   
80  0.217500  0.4455  0.583250  52.70  ...  6.300000  10.200000   
81  0.132333  0.5240  0.660333  48.70  ...  5.033333   8.633333   

   Offensive Four Factors                              Defensive Four Factors   
                     eFG%    TOV%       ORB%    FT/FGA                   eFG%   
0                0.634000   9.600  18.200000  0.268000               0.581000  \
1                0.608000  18.200  32.400000  0.190000               0.500000   
2                0.592000   7.500  16.300000  0.264000               0.553000   
3                0.460000   8.700  23.300000  0.100000               0.540000   
4                0.591000  16.500  13.200000  0.317000               0.609000   
..                    ...     ...        ...       ...                    ...   
77               0.564000   7.650  15.900000  0.172000               0.523500   
78               0.594500  10.650  20.825000  0.140750               0.541000   
79               0.530333  12.500  24.833333  0.148000               0.512667   
80               0.559750  13.725  27.650000  0.160500               0.504000   
81               0.643333  11.400  18.266667  0.110333               0.503333   

                                    
         TOV%       DRB%    FT/FGA  
0   13.200000  88.200000  0.300000  
1    8.400000  73.800000  0.130000  
2    7.900000  71.400000  0.158000  
3   11.500000  68.900000  0.299000  
4   12.800000  76.300000  0.299000  
..        ...        ...       ...  
77  10.600000  69.200000  0.155000  
78   9.875000  83.975000  0.254750  
79  11.800000  77.433333  0.153667  
80  10.475000  77.300000  0.150250  
81   8.066667  72.300000  0.134667  

[82 rows x 21 columns]

In [43]:
TGL_STATS_H2H_CUMU_STD_DF

Result                         Advanced                                  
         W/L         Tm        Opp       ORtg       DRtg      Pace       FTr   
0        NaN        NaN        NaN        NaN        NaN       NaN       NaN  \
1        NaN        NaN        NaN        NaN        NaN       NaN       NaN   
2        NaN        NaN        NaN        NaN        NaN       NaN       NaN   
3        NaN        NaN        NaN        NaN        NaN       NaN       NaN   
4        NaN        NaN        NaN        NaN        NaN       NaN       NaN   
..       ...        ...        ...        ...        ...       ...       ...   
77  0.707107   3.535534   2.828427   4.949747  11.172287  7.000357  0.033234   
78  0.500000  10.812801   7.724420   5.806605   2.891943  4.901275  0.097725   
79  0.000000   9.504385   8.621678   9.377811   8.719518  0.953939  0.028537   
80  0.000000  10.677078   7.041543  11.677436   7.164263  1.152895  0.036171   
81  0.000000   7.023769  12.013881   3.384277  11.150486  2.956913  0.022189   

                                  ...                       
        3PAr       TS%      TRB%  ...      STL%      BLK%   
0        NaN       NaN       NaN  ...       NaN       NaN  \
1        NaN       NaN       NaN  ...       NaN       NaN   
2        NaN       NaN       NaN  ...       NaN       NaN   
3        NaN       NaN       NaN  ...       NaN       NaN   
4        NaN       NaN       NaN  ...       NaN       NaN   
..       ...       ...       ...  ...       ...       ...   
77  0.034648  0.027577  2.121320  ...  3.959798  1.697056   
78  0.034809  0.070315  1.391642  ...  2.495830  1.276388   
79  0.013528  0.047120  1.311488  ...  2.250185  3.931073   
80  0.065932  0.069462  5.113381  ...  2.270095  3.292416   
81  0.082505  0.026350  3.304542  ...  0.960902  3.225420   

   Offensive Four Factors                                 
                     eFG%      TOV%      ORB%    FT/FGA   
0                     NaN       NaN       NaN       NaN  \
1                     NaN       NaN       NaN       NaN   
2                     NaN       NaN       NaN       NaN   
3                     NaN       NaN       NaN       NaN   
4                     NaN       NaN       NaN       NaN   
..                    ...       ...       ...       ...   
77               0.053740  2.474874  6.363961  0.066468   
78               0.067540  2.738004  6.315787  0.085234   
79               0.037166  2.193171  2.532456  0.031193   
80               0.066199  3.034661  6.000833  0.035688   
81               0.027791  3.469870  7.031595  0.011930   

   Defensive Four Factors                                
                     eFG%      TOV%      DRB%    FT/FGA  
0                     NaN       NaN       NaN       NaN  
1                     NaN       NaN       NaN       NaN  
2                     NaN       NaN       NaN       NaN  
3                     NaN       NaN       NaN       NaN  
4                     NaN       NaN       NaN       NaN  
..                    ...       ...       ...       ...  
77               0.003536  6.222540  2.687006  0.028284  
78               0.040620  2.928452  2.937544  0.089615  
79               0.066161  1.752142  5.105226  0.100848  
80               0.056733  3.011506  4.176921  0.082625  
81               0.050332  0.378594  3.439477  0.030989  

[82 rows x 21 columns]

In [ ]:
# Convert this into a function that can be applied to a series. 
# Compute consecutive home/away count
def __compute_venue_streak_count__(series):
    idx_sequence = []
    count = 0
    for i in range(len(series)):
        if i == 0 or series[i] != series[i-1]:
            count = 0
        else:
            count += 1
        idx_sequence.append(count)
    return idx_sequence

__compute_venue_streak_count__(TGL_DF[('Match','H/A')])

In [ ]:
def __compute_days_between_rows__(df,window=2):
    df = pd.to_datetime(df.copy())
    return df.diff(window-1).dt.days # -1 because diff() computes the difference between the current and the previous row

def __compute_cumulative_avg_stats__(df):
    return df.expanding().mean(),df.expanding().std()

def __compute_rolling_avg_stats__(df, window=5):
    return df.rolling(window).mean(),df.rolling(window).std()

def __compute_pts_total__(df):
    return df.sum(axis=1)

def __compute_pts_spread__(df):
    return df.apply(lambda x: x.iloc[0] - x.iloc[1],axis=1)


def compute_gamelog_stats_regular_season():
    """
    Description:
        Compute gamelog stats
    Summary:
        1. Load all gamelog csv
        2. Compute the expanding and rolling stats
        3. Save the computed gamelogs csv
    Usage:
        python3 03_compute_script.py -m gamelogs_stats -s ../data-parsed/ -t ../data-computed/
    
    """
    _FAILS_ = []
    # Get the list of computed gamelog files
    # TGT_TGL_CSV_COMP_TXT = f'{TGT_DIR}/tgl_csv_stats_computed.txt'
    # TGT_TGL_CSV_COMP_LIST = []
    # if file_exists(TGT_TGL_CSV_COMP_TXT):
    #     TGT_TGL_CSV_COMP_LIST = load_file(TGT_TGL_CSV_COMP_TXT).split('\n')

    # Get all gamelog files
    SRC_TGL_CSV_LIST = get_all_files_recursive(f'{SRC_DIR}/teams', file_type=('tgl_basic.csv','tgl_advanced.csv')) # get only regular season gamelog files
    if debug:
        SRC_TGL_CSV_LIST = SRC_TGL_CSV_LIST[:300]

    TQDM_SRC_TGL_CSV_LIST = tqdm(SRC_TGL_CSV_LIST,ncols=150,dynamic_ncols=False)
    for SRC_TGL_CSV in TQDM_SRC_TGL_CSV_LIST:
        TQDM_SRC_TGL_CSV_LIST.set_description(SRC_TGL_CSV)       
        try:
            # Load the gamelog csv
            TGL_DF = pd.read_csv(SRC_TGL_CSV,header=[0,1])
            TGL_DF = TGL_DF.drop(columns=[('Unnamed: 0_level_0','Unnamed: 0_level_1')])
            # Compute the expanding and rolling stats
            TGL_MDATE_DF = TGL_DF[('Match','Date')]
            TGL_VEN_STRK_CNT = __compute_venue_streak_count__(TGL_DF[('Match','H/A')]).rename(('Match','Venue_Strk_Cnt'))
            TGL_DAYS_BTW_2GM = __compute_days_between_rows__(TGL_MDATE_DF,window=2).rename(('Match','Days_Btw_2GM'))
            TGL_DAYS_BTW_3GM = __compute_days_between_rows__(TGL_MDATE_DF,window=3).rename(('Match','Days_Btw_3GM'))
            TGL_DAYS_BTW_4GM = __compute_days_between_rows__(TGL_MDATE_DF,window=4).rename(('Match','Days_Btw_4GM'))
            TGL_DAYS_BTW_GM_DF = pd.concat([TGL_DAYS_BTW_2GM,TGL_DAYS_BTW_3GM,TGL_DAYS_BTW_4GM,TGL_VEN_STRK_CNT],axis=1) 
            # Compute the total points, spread points
            TGL_RESULT_DF = TGL_DF['Result'].copy()
            TGL_PTS_TOTAL = __compute_pts_total__(TGL_RESULT_DF[['Tm','Opp']]).rename('Pts_Total')
            TGL_PTS_SPREAD = __compute_pts_spread__(TGL_RESULT_DF[['Tm','Opp']]).rename('Pts_Spread')
            TGL_RESULT_DF = pd.concat([TGL_RESULT_DF,TGL_PTS_TOTAL,TGL_PTS_SPREAD],axis=1)
            # Compute the expanding and rolling stats
            TGL_STATS_DF = TGL_DF.drop('Match',level=0,axis=1)
            TGL_STATS_CUMU_AVG_DF,TGL_STATS_CUMU_STD_DF        = __compute_cumulative_avg_stats__(TGL_STATS_DF)
            TGL_STATS_ROLL_04_AVG_DF, TGL_STATS_ROLL_04_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=4)
            TGL_STATS_ROLL_08_AVG_DF, TGL_STATS_ROLL_08_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=8)
            TGL_STATS_ROLL_12_AVG_DF, TGL_STATS_ROLL_12_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=12)
            TGL_STATS_ROLL_16_AVG_DF, TGL_STATS_ROLL_16_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=16)
            TGL_STATS_ROLL_20_AVG_DF, TGL_STATS_ROLL_20_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=20)
            # Compute the home/away stats
            TGL_STATS_AW_DF = TGL_DF[TGL_DF[('Match','H/A')] == 0].drop('Match',level=0,axis=1)
            TGL_STATS_HM_DF = TGL_DF[TGL_DF[('Match','H/A')] == 1].drop('Match',level=0,axis=1)
            TGL_STATS_AW_CUMU_AVG_DF, TGL_STATS_AW_CUMU_STD_DF      = __compute_cumulative_avg_stats__(TGL_STATS_AW_DF)
            TGL_STATS_AW_ROLL_04_AVG_DF,TGL_STATS_AW_ROLL_04_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_AW_DF,window=4)
            TGL_STATS_AW_ROLL_08_AVG_DF,TGL_STATS_AW_ROLL_08_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_AW_DF,window=8)
            TGL_STATS_AW_ROLL_12_AVG_DF,TGL_STATS_AW_ROLL_12_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_AW_DF,window=12)
            TGL_STATS_AW_ROLL_16_AVG_DF,TGL_STATS_AW_ROLL_16_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_AW_DF,window=16)
            TGL_STATS_AW_ROLL_20_AVG_DF,TGL_STATS_AW_ROLL_20_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_AW_DF,window=20)
            TGL_STATS_HM_CUMU_AVG_DF, TGL_STATS_HM_CUMU_STD_DF      = __compute_cumulative_avg_stats__(TGL_STATS_HM_DF)
            TGL_STATS_HM_ROLL_04_AVG_DF,TGL_STATS_HM_ROLL_04_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_HM_DF,window=4)
            TGL_STATS_HM_ROLL_08_AVG_DF,TGL_STATS_HM_ROLL_08_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_HM_DF,window=8)
            TGL_STATS_HM_ROLL_12_AVG_DF,TGL_STATS_HM_ROLL_12_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_HM_DF,window=12)
            TGL_STATS_HM_ROLL_16_AVG_DF,TGL_STATS_HM_ROLL_16_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_HM_DF,window=16)
            TGL_STATS_HM_ROLL_20_AVG_DF,TGL_STATS_HM_ROLL_20_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_HM_DF,window=20)
            # Compute the h2h stats
            TGL_STATS_H2H_CUMU_AVG_DF, TGL_STATS_H2H_CUMU_STD_DF    = __compute_h2h_avg_stats__(TGL_DF)

            # Save the computed gamelogs csv
            TGL_DICT = {
                'mday_rest_days' : TGL_DAYS_BTW_GM_DF,   'mday_gm_results'   : TGL_RESULT_DF,
                'stats_cumu_avg'    : TGL_STATS_CUMU_AVG_DF,    'stats_cumu_std'    : TGL_STATS_CUMU_STD_DF,
                'stats_roll_04_avg' : TGL_STATS_ROLL_04_AVG_DF, 'stats_roll_04_std' : TGL_STATS_ROLL_04_STD_DF,
                'stats_roll_08_avg' : TGL_STATS_ROLL_08_AVG_DF, 'stats_roll_08_std' : TGL_STATS_ROLL_08_STD_DF,
                'stats_roll_12_avg' : TGL_STATS_ROLL_12_AVG_DF, 'stats_roll_12_std' : TGL_STATS_ROLL_12_STD_DF,
                'stats_roll_16_avg' : TGL_STATS_ROLL_16_AVG_DF, 'stats_roll_16_std' : TGL_STATS_ROLL_16_STD_DF,
                'stats_roll_20_avg' : TGL_STATS_ROLL_20_AVG_DF, 'stats_roll_20_std' : TGL_STATS_ROLL_20_STD_DF,
                'stats_hm_cumu_avg' : TGL_STATS_HM_CUMU_AVG_DF, 'stats_hm_cumu_std' : TGL_STATS_HM_CUMU_STD_DF,
                'stats_hm_roll_04_avg' : TGL_STATS_HM_ROLL_04_AVG_DF, 'stats_hm_roll_04_std' : TGL_STATS_HM_ROLL_04_STD_DF,
                'stats_hm_roll_08_avg' : TGL_STATS_HM_ROLL_08_AVG_DF, 'stats_hm_roll_08_std' : TGL_STATS_HM_ROLL_08_STD_DF,
                'stats_hm_roll_12_avg' : TGL_STATS_HM_ROLL_12_AVG_DF, 'stats_hm_roll_12_std' : TGL_STATS_HM_ROLL_12_STD_DF,
                'stats_hm_roll_16_avg' : TGL_STATS_HM_ROLL_16_AVG_DF, 'stats_hm_roll_16_std' : TGL_STATS_HM_ROLL_16_STD_DF,
                'stats_hm_roll_20_avg' : TGL_STATS_HM_ROLL_20_AVG_DF, 'stats_hm_roll_20_std' : TGL_STATS_HM_ROLL_20_STD_DF,
                'stats_aw_cumu_avg' : TGL_STATS_AW_CUMU_AVG_DF, 'stats_aw_cumu_std' : TGL_STATS_AW_CUMU_STD_DF,
                'stats_aw_roll_04_avg' : TGL_STATS_AW_ROLL_04_AVG_DF, 'stats_aw_roll_04_std' : TGL_STATS_AW_ROLL_04_STD_DF,
                'stats_aw_roll_08_avg' : TGL_STATS_AW_ROLL_08_AVG_DF, 'stats_aw_roll_08_std' : TGL_STATS_AW_ROLL_08_STD_DF,
                'stats_aw_roll_12_avg' : TGL_STATS_AW_ROLL_12_AVG_DF, 'stats_aw_roll_12_std' : TGL_STATS_AW_ROLL_12_STD_DF,
                'stats_aw_roll_16_avg' : TGL_STATS_AW_ROLL_16_AVG_DF, 'stats_aw_roll_16_std' : TGL_STATS_AW_ROLL_16_STD_DF,
                'stats_aw_roll_20_avg' : TGL_STATS_AW_ROLL_20_AVG_DF, 'stats_aw_roll_20_std' : TGL_STATS_AW_ROLL_20_STD_DF,
                'stats_h2h_cumu_avg'   : TGL_STATS_H2H_CUMU_AVG_DF, 'stats_h2h_cumu_std': TGL_STATS_H2H_CUMU_STD_DF,
            }
            # Extract the team and season from the directory
            TGL_STATS_DIR = SRC_TGL_CSV.replace(SRC_DIR,'').replace('.csv','')
            make_directory(f'{TGT_DIR}/{TGL_STATS_DIR}')
            for TBL_ID, DF in TGL_DICT.items():
                DF.to_csv(f'{TGT_DIR}/{TGL_STATS_DIR}/{TBL_ID}.csv', index=False)
            
        except Exception as e:
            print(f'Error computing stats for {SRC_TGL_CSV}: {e}')
            _FAILS_.append(SRC_TGL_CSV)
            continue
    if _FAILS_:
        print(f'Failed to compute stats for {len(_FAILS_)}/{len(SRC_TGL_CSV_LIST)} files: {_FAILS_}')
    else:
        print(f'All {len(SRC_TGL_CSV_LIST)} files computed successfully')

if __name__ == '__main__':
    if MODE == 'gamelogs_stats':
        compute_gamelog_stats_regular_season()
    else:
        raise ValueError(f'Invalid mode: {MODE}')

    
